In [1]:
'''Grammar Correction with Neuspell'''
import neuspell
from neuspell import BertChecker

# load the model from the checkpoint
checker = BertChecker()
checker.vocab_path = '/bigpool/homes/chatterjee/anaconda3/envs/e2ecoref/lib/python3.7/site-packages/neuspell/subwordbert-probwordnoise/vocab.pkl'
checker.from_pretrained('/bigpool/homes/chatterjee/anaconda3/envs/e2ecoref/lib/python3.7/site-packages/neuspell/subwordbert-probwordnoise/')

data folder is set to `/bigpool/homes/chatterjee/anaconda3/envs/e2ecoref/lib/python3.7/site-packages/neuspell/../data` script


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

loading vocab from path:/bigpool/homes/chatterjee/anaconda3/envs/e2ecoref/lib/python3.7/site-packages/neuspell/subwordbert-probwordnoise/vocab.pkl
initializing model


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SubwordBert(
  (bert_dropout): Dropout(p=0.2, inplace=False)
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [20]:
checker.correct('I lik to play footbal')

'I like to play football'

In [23]:
checker.correct('I will eat fish for dinner and drank milk')

'I will eat fish for dinner and drink milk'

In [3]:
'''Load the three question subsets'''
def load_questions(path):
    with open(path, 'r') as f:
        questions = f.readlines()
        questions = [x.strip() for x in questions] 
    return questions

hundred_percent = load_questions('../embeddings/questions/100/questions.txt')
ten_percent = load_questions('../embeddings/questions/10/questions.txt')
one_percent = load_questions('../embeddings/questions/1/questions.txt')

In [11]:
# for the one_percent subset, correct the grammar and save the corrected questions for the LAL-Parser
from tqdm import tqdm

def correct_grammar(questions):
    corrected = []
    count = 0
    for q in tqdm(questions):
        correct = checker.correct(q)
        # remove the space before the question mark
        correct = correct.replace(' ?', '?')
        corrected.append(correct)
        if q != correct:
            count += 1
    return corrected, count

corrected_one_percent, count = correct_grammar(one_percent)

100%|██████████| 8402/8402 [04:37<00:00, 30.24it/s]


In [12]:
one_percent[:5]

['is banana next to glass?',
 'is he wearing all white?',
 'what does pet bed look like?',
 'how old is the person holding the kite?',
 'what is the age of the men?']

In [13]:
corrected_one_percent[:5]

['is bananas next to glass?',
 'is he wearing all white?',
 'what does pet bed look like?',
 'how old is the person holding the kite?',
 'what is the age of the men?']

In [14]:
count

1010

In [17]:
len(corrected_one_percent)

8402

In [15]:
with open('../embeddings/questions/1/questions_corrected.txt', 'w') as f:
    for item in corrected_one_percent:
        # write each sentence to a new line
        f.write("%s\n" % item)

In [16]:
# load the corrected questions
test = load_questions('../embeddings/questions/1/questions_corrected.txt')
test[:5]

['is bananas next to glass?',
 'is he wearing all white?',
 'what does pet bed look like?',
 'how old is the person holding the kite?',
 'what is the age of the men?']